In [3]:
import json
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [4]:
with open("../../datasets/export-lindat/lindat-20251031_json/metadata_20250431.json", "r") as f:
    meta = pd.DataFrame(json.load(f)).set_index("screening_id")

target_map = {0: 0, 2: 1, 3: 1}  # map targets: 0 -> 0 and (2, 3) -> 1
raget_key = "kobar_kategorizace_definitivni"
targets = meta.loc[meta[raget_key].isin(target_map), raget_key].map(target_map).sort_index()

with open("../../datasets/export-lindat/lindat-20251031_json/expert_features_zipformer_lm-extra06.json", "r") as f:
    features = pd.DataFrame(json.load(f)).set_index("screening_id")
    features = features.loc[features.index.intersection(targets.index)].sort_index()

with open("../../datasets/export-lindat/lindat-20251031_json/train_20250431.json", "r") as f:
    train_ids = json.load(f)
with open("../../datasets/export-lindat/lindat-20251031_json/test_20250431.json", "r") as f:
    test_ids = json.load(f)

X_train = features.loc[features.index.isin(train_ids)]
y_train = targets.loc[targets.index.isin(train_ids)]

X_test = features.loc[features.index.isin(test_ids)]
y_test = targets.loc[targets.index.isin(test_ids)]

FileNotFoundError: [Errno 2] No such file or directory: '../../datasets/export-lindat/lindat-20251031_json/metadata_20250431.json'

In [2]:
model = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
        ("logreg", LogisticRegression(random_state=42, max_iter=1000, solver="liblinear")),
    ]
)

NameError: name 'Pipeline' is not defined

In [ ]:
features = []

X_train_task = X_train[features]
y_train_task = y_train.loc[y_train.index.intersection(X_train_task.index)]

X_test_task = X_test[features]
y_test_task = y_test.loc[y_test.index.intersection(X_test_task.index)]

model.fit(X_train_task, y_train_task)
predicted = model.predict(X_test_task)
report = classification_report(y_test_task, predicted, zero_division=0.0)
print(report)